In [53]:
import sympy as sp
import numpy as np
# from sympy import *

# ----- Symbols -----
alpha, beta, psi = sp.symbols('alpha beta psi', real=True)   # GW direction & pol angle
omega = sp.symbols('omega', positive=True, real=True)        # GW angular frequency

# Lab/PD spatial coordinates and vectors
x, y, z = sp.symbols('x y z', real=True)
t = sp.symbols('t', real=True)
r, phi = sp.symbols('r phi', real=True)  # cylindrical coords
X = sp.Matrix([x, y, z])

n_x, n_y, n_z = sp.symbols('n_x n_y n_z', real=True)
n = sp.Matrix([n_x, n_y, n_z])

Q = sp.symbols('Q', positive=True)
F0, F1, F2 = sp.symbols('F0 F1 F2', real=True)
F0p, F1p, F2p = sp.symbols('F0p F1p F2p', real=True)

# Strain amplitudes (time-dependent, keep symbolic)
h_plus  = sp.symbols('h_+')
h_cross = sp.symbols('h_\\times')

# helicity basis
h_p2, h_m2 = sp.symbols('h_+2 h_-2')

# # Spherical tangent basis at n̂
# th = sp.Matrix([sp.cos(beta)*sp.cos(alpha), sp.cos(beta)*sp.sin(alpha), -sp.sin(beta)])
# ph = sp.Matrix([-sp.sin(alpha),             sp.cos(alpha),              0])

# # Rotate by polarization angle ψ about n̂ to get (p,q)
# p =  th*sp.cos(psi) + ph*sp.sin(psi)
# q = -th*sp.sin(psi) + ph*sp.cos(psi)

# # ----- Polarization tensors in lab/PD basis -----
# e_plus  = sp.Matrix([[p[i]*p[j] - q[i]*q[j] for j in range(3)] for i in range(3)])
# e_cross = sp.Matrix([[p[i]*q[j] + q[i]*p[j] for j in range(3)] for i in range(3)])

exx_p, exy_p, exz_p, eyy_p, eyz_p, ezz_p = sp.symbols('e_xx^+ e_xy^+ e_xz^+ e_yy^+ e_yz^+ e_zz^+', real=True)
exx_c, exy_c, exz_c, eyy_c, eyz_c, ezz_c = sp.symbols('e_xx^\\times e_xy^\\times e_xz^\\times' \
'                                                    e_yy^\\times e_yz^\\times e_zz^\\times', real=True)

e_plus = sp.Matrix([[exx_p, exy_p, exz_p],
                   [exy_p, eyy_p, eyz_p],
                  [exz_p, eyz_p, ezz_p]])

e_cross = sp.Matrix([[exx_c, exy_c, exz_c],
                    [exy_c, eyy_c, eyz_c],
                    [exz_c, eyz_c, ezz_c]])

# Combine pols with (symbolic) strains
hTT = (h_plus*e_plus + h_cross*e_cross)*sp.exp(1j*omega*t)

# ----- Useful substitutions -----
berlin = {alpha:0, beta:0, psi:0,
          n_x:0, n_y:0, n_z:1,
          exx_p:1,eyy_p:-1,ezz_p:0,exy_p:0,exz_p:0,eyz_p:0,
          exx_c:0,eyy_c:0,ezz_c:0,exy_c:1,exz_c:0,eyz_c:0} # hTT_ij(i,j).subs(berlin) gives on axis GW result in Berlin

cart2cyl = {x:r*sp.cos(phi), y:r*sp.sin(phi), z:z}

f0_Q = -1j/Q + (1-sp.exp(-1j*Q))/Q**2
f1_Q = -1j/(2*Q) - sp.exp(-1j*Q)/Q**2 - 1j*(1-sp.exp(-1j*Q))/Q**3
f2_Q = -(1+sp.exp(-1j*Q))/Q**2 - 2j*(1-sp.exp(-1j*Q))/Q**3
f0p_Q = sp.diff(f0_Q, Q)
f1p_Q = sp.diff(f1_Q, Q)
f2p_Q = sp.diff(f2_Q, Q)
expand_fq = {F0: f0_Q, F1: f1_Q, F2: f2_Q, F0p: f0p_Q, F1p: f1p_Q, F2p: f2p_Q}

write_in_hel_basis = {h_plus:(h_p2+h_m2)/sp.sqrt(2), h_cross:-1j*(h_p2-h_m2)/sp.sqrt(2)}




# helpers
ndotx = Q/omega
hx    = hTT*X
nhx   = n.dot(hx)

def hTT_ij(i, j):
    return hTT[i-1, j-1]
def hTT_ijxj(i):
    return hx[i-1]
def ni_hTT_ij(j):
    return n[0]*hTT_ij(1, j) + n[1]*hTT_ij(2, j) + n[2]*hTT_ij(3, j)

# h^PD kernels
S = hx.dot(X)
def V_i(i):
    ni = n[i-1]
    return ndotx*hTT_ijxj(i) - ni*S
def T_ij(i, j):
    ni = n[i-1]
    nj = n[j-1]
    return ndotx*nj*hTT_ijxj(i)+ndotx*ni*hTT_ijxj(j)-ni*nj*S-(ndotx**2)*hTT_ij(i, j)

h00 = -S * F0
def h0i(i):
    return -omega*omega * V_i(i) * F1

def hij(i, j):
    return omega*omega * T_ij(i, j) * F2

hmat = sp.Matrix([[h00, h0i(1), h0i(2), h0i(3)],
                 [h0i(1), hij(1,1), hij(1,2), hij(1,3)],
                 [h0i(2), hij(2,1), hij(2,2), hij(2,3)],
                 [h0i(3), hij(3,1), hij(3,2), hij(3,3)]])

hmat_one_up = sp.Matrix([[-h00, -h0i(1), -h0i(2), -h0i(3)],
                 [-h0i(1), hij(1,1), hij(1,2), hij(1,3)],
                 [-h0i(2), hij(2,1), hij(2,2), hij(2,3)],
                 [-h0i(3), hij(3,1), hij(3,2), hij(3,3)]])

Fmat = sp.Matrix([[0, 0, 0, 0],
                 [0, 0, 1, 0],
                 [0, -1, 0, 0],
                 [0, 0, 0, 0]])

h = sp.trace(hmat_one_up)

def dmS(m):
    return 2*hTT_ijxj(m)

def dmV_i(m, i):
    nm = n[m-1]
    ni = n[i-1]
    return ndotx*hTT_ij(i,m) - 2*ni*hTT_ijxj(m) + nm*hTT_ijxj(i)

def dmT_ij(m, i, j):
    nm = n[m-1]
    ni = n[i-1]
    nj = n[j-1]
    return ndotx*(ni*hTT_ij(j,m)+nj*hTT_ij(i,m)-2*nm*hTT_ij(i,j)) + nm*(nj*hTT_ijxj(i)+ni*hTT_ijxj(j)) - 2*ni*nj*hTT_ijxj(m)

# partial derivatives of PD frame h_uv components and trace

def d0h_0i(i):
    # ni = n[i-1]
    return 1j*omega*(omega**2*V_i(i)*F1)

def dmh_00(m):
    nm = n[m-1]
    return omega**2 * (F0*dmS(m) + S*F0p*nm*omega)

def dmh_0i(m, i):
    nm = n[m-1]
    # ni = n[i-1]
    return omega**2 * (F1*dmV_i(m, i) + V_i(i)*F1p*nm*omega)

def dmh_ij(m, i, j):
    nm = n[m-1]
    # ni = n[i-1]
    # nj = n[j-1]
    return omega**2 * (F2*dmT_ij(m, i, j) + T_ij(i, j)*F2p*nm*omega)

def dmh_trace(m):
    return -dmh_00(m) + dmh_ij(m,1,1) + dmh_ij(m,2,2) + dmh_ij(m,3,3)

In [54]:
h.subs(berlin).subs(expand_fq)

-omega**2*((-1 - exp(-1.0*I*Q))/Q**2 - 2.0*I*(1 - exp(-1.0*I*Q))/Q**3)*(x*(h_+*x*exp(1.0*I*omega*t) + h_\times*y*exp(1.0*I*omega*t)) + y*(-h_+*y*exp(1.0*I*omega*t) + h_\times*x*exp(1.0*I*omega*t))) - (-1.0*I/Q + (1 - exp(-1.0*I*Q))/Q**2)*(-x*(h_+*x*exp(1.0*I*omega*t) + h_\times*y*exp(1.0*I*omega*t)) - y*(-h_+*y*exp(1.0*I*omega*t) + h_\times*x*exp(1.0*I*omega*t)))

In [63]:
def hF(m, n):
    return hmat_one_up[m,0]*Fmat[0,n] + hmat_one_up[m,1]*Fmat[1,n] + hmat_one_up[m,2]*Fmat[2,n] + hmat_one_up[m,3]*Fmat[3,n]

def jmn(m, n):
    # j^\mu = \partial_\nv j^{\mu\nv}
    return 0.5*h*Fmat[m,n] + hF(n, m) - hF(m, n)






rho = sp.diff(jmn(0,0),t) + sp.diff(jmn(0,1),x) + sp.diff(jmn(0,2),y) + sp.diff(jmn(0,3),z)
jx = sp.diff(jmn(1,0),t) + sp.diff(jmn(1,1),x) + sp.diff(jmn(1,2),y) + sp.diff(jmn(1,3),z)
jy = sp.diff(jmn(2,0),t) + sp.diff(jmn(2,1),x) + sp.diff(jmn(2,2),y) + sp.diff(jmn(2,3),z)
jz = sp.diff(jmn(3,0),t) + sp.diff(jmn(3,1),x) + sp.diff(jmn(3,2),y) + sp.diff(jmn(3,3),z)

jr = jx*sp.cos(phi) + jy*sp.sin(phi)
jphi = -jx*sp.sin(phi) + jy*sp.cos(phi)

In [88]:
f_factor = (jphi*sp.exp(-2j*phi)/r/omega/omega).subs(berlin).subs(cart2cyl).subs(write_in_hel_basis).subs({t: 0}).rewrite(sp.exp).expand().collect([h_p2, h_m2]).coeff(h_m2)
f_factor

-0.5*sqrt(2)*F0/omega**2 + 0.5*sqrt(2)*I*F1*Q + 0.5*sqrt(2)*F2

In [ ]:
f_factor.subs(expand_fq).expand().col

0.25*sqrt(2) - 0.5*sqrt(2)*I*exp(-1.0*I*Q)/Q + 0.5*sqrt(2)*I/(Q*omega**2) - 1.0*sqrt(2)*exp(-1.0*I*Q)/Q**2 - 0.5*sqrt(2)/(Q**2*omega**2) + 0.5*sqrt(2)*exp(-1.0*I*Q)/(Q**2*omega**2) - 1.0*sqrt(2)*I/Q**3 + 1.0*sqrt(2)*I*exp(-1.0*I*Q)/Q**3

Symbolic J0

In [2]:
j0_over_Bw2 = ((dmh_0i(2,1)-dmh_0i(1,2))/omega**2).expand().subs(cart2cyl).collect([h_plus, h_cross])
j0_p = j0_over_Bw2.coeff(h_plus).collect([F1*Q, 3*F1, F1p*Q])
j0_c = j0_over_Bw2.coeff(h_cross).collect([F1*Q, 3*F1, F1p*Q])

In [3]:
j0_p = j0_p.simplify()
j0_c = j0_c.simplify()

j0_p

(3*F1 + F1p*Q)*(e_xx^+*n_y*r*cos(phi) - e_xy^+*n_x*r*cos(phi) + e_xy^+*n_y*r*sin(phi) + e_xz^+*n_y*z - e_yy^+*n_x*r*sin(phi) - e_yz^+*n_x*z)

Symbolic Jx, Jy

In [4]:
jx_over_Bw2 = ((0.5*dmh_trace(2)+d0h_0i(2)-dmh_ij(2,2,2)-dmh_ij(3,3,2)-dmh_ij(2,1,1))/omega**2)         \
                .subs(cart2cyl).expand().collect([h_plus, h_cross])
jy_over_Bw2 = ((-0.5*dmh_trace(1)-d0h_0i(1)+dmh_ij(1,1,1)+dmh_ij(3,3,1)+dmh_ij(1,2,2))/omega**2)    \
                .subs(cart2cyl).expand().collect([h_plus, h_cross])

(Jx, Jy) -> (Jr, Jphi)

In [5]:
jr_over_Bw2 = jx_over_Bw2*sp.cos(phi) + jy_over_Bw2*sp.sin(phi)
jphi_over_Bw2 = -jx_over_Bw2*sp.sin(phi) + jy_over_Bw2*sp.cos(phi)

jr_p = jr_over_Bw2.expand().coeff(h_plus).collect([F1*Q,F1,F2*Q,F2,F2p,F2p*Q,F2p*Q*Q]).collect([exx_p, 
            exy_p, exz_p, eyy_p, eyz_p, ezz_p])
jr_c = jr_over_Bw2.expand().coeff(h_cross).collect([F1*Q,F1,F2*Q,F2,F2p,F2p*Q,F2p*Q*Q]).collect([exx_c, 
            exy_c, exz_c, eyy_c, eyz_c, ezz_c])

jphi_p = jphi_over_Bw2.expand().coeff(h_plus).collect([F1*Q,F1,F2*Q,F2,F2p,F2p*Q,F2p*Q*Q]).collect([exx_p, 
            exy_p, exz_p, eyy_p, eyz_p, ezz_p])
jphi_c = jphi_over_Bw2.expand().coeff(h_cross).collect([F1*Q,F1,F2*Q,F2,F2p,F2p*Q,F2p*Q*Q]).collect([exx_c, 
            exy_c, exz_c, eyy_c, eyz_c, ezz_c])

In [6]:
jr_p = jr_p.expand().collect([exx_p, eyy_p, exy_p, exz_p, eyz_p, ezz_p])
jr_c = jr_c.expand().collect([exx_c, eyy_c, exy_c, exz_c, eyz_c, ezz_c])
jphi_p = jphi_p.expand().collect([exx_p, eyy_p, exy_p, exz_p, eyz_p, ezz_p])
jphi_c = jphi_c.expand().collect([exx_c, eyy_c, exy_c, exz_c, eyz_c, ezz_c])

In [ ]:
jr_p

e_xx^+*(1.0*F0*r*sin(phi)*cos(phi) + 0.5*F0p*n_x*omega*r**2*sin(phi)*cos(phi)**2 - 0.5*F0p*n_y*omega*r**2*cos(phi)**3 - 1.0*I*F1*Q*r*sin(phi)*cos(phi) + 1.0*I*F1*n_x*omega*r**2*sin(phi)*cos(phi)**2 - 1.0*I*F1*n_y*omega*r**2*cos(phi)**3 + 1.0*F2*Q*n_y*cos(phi)/omega - 1.0*F2*n_x*n_y*r*cos(phi)**2 - 1.0*F2*n_y**2*r*sin(phi)*cos(phi) + 2.0*F2*n_z**2*r*sin(phi)*cos(phi) - 0.5*F2p*Q**2*n_x*sin(phi)/omega + 0.5*F2p*Q**2*n_y*cos(phi)/omega + 1.0*F2p*Q*n_x**2*r*sin(phi)*cos(phi) - 1.0*F2p*Q*n_x*n_y*r*cos(phi)**2 + F2p*Q*n_z**2*r*sin(phi)*cos(phi) - 0.5*F2p*n_x**3*omega*r**2*sin(phi)*cos(phi)**2 + 0.5*F2p*n_x**2*n_y*omega*r**2*cos(phi)**3 - 0.5*F2p*n_x*n_y**2*omega*r**2*sin(phi)*cos(phi)**2 - 0.5*F2p*n_x*n_z**2*omega*r**2*sin(phi)*cos(phi)**2 + 0.5*F2p*n_y**3*omega*r**2*cos(phi)**3 + 0.5*F2p*n_y*n_z**2*omega*r**2*cos(phi)**3) + e_xy^+*(1.0*F0*r*sin(phi)**2 - 1.0*F0*r*cos(phi)**2 + 1.0*F0p*n_x*omega*r**2*sin(phi)**2*cos(phi) - 1.0*F0p*n_y*omega*r**2*sin(phi)*cos(phi)**2 - 1.0*I*F1*Q*r*sin(phi)**

In [8]:
jr_c.subs(berlin)

1.0*F0*r*sin(phi)**2 - 1.0*F0*r*cos(phi)**2 - 1.0*I*F1*Q*r*sin(phi)**2 + 1.0*I*F1*Q*r*cos(phi)**2 + 2.0*F2*r*sin(phi)**2 - 2.0*F2*r*cos(phi)**2 + F2p*Q*r*sin(phi)**2 - F2p*Q*r*cos(phi)**2

In [9]:
jphi_p.subs(berlin)

-1.0*F0*r*sin(phi)**2 + 1.0*F0*r*cos(phi)**2 + 1.0*I*F1*Q*r*sin(phi)**2 - 1.0*I*F1*Q*r*cos(phi)**2 - 2.0*F2*r*sin(phi)**2 + 2.0*F2*r*cos(phi)**2 - F2p*Q*r*sin(phi)**2 + F2p*Q*r*cos(phi)**2

In [10]:
jphi_c.subs(berlin)

2.0*F0*r*sin(phi)*cos(phi) - 2.0*I*F1*Q*r*sin(phi)*cos(phi) + 4.0*F2*r*sin(phi)*cos(phi) + 2*F2p*Q*r*sin(phi)*cos(phi)

Helicity basis (Berlin eq 24):

In [11]:
jr_berlin = (-6*sp.sqrt(2)*jr_over_Bw2).subs(write_in_hel_basis).expand().rewrite(sp.exp).expand().powsimp().collect([h_p2, h_m2]).subs(berlin)
jr_berlin

h_+2*(-6.0*I*F0*r*exp(-2*I*phi) - 6.0*F1*Q*r*exp(-2*I*phi) - 12.0*I*F2*r*exp(-2*I*phi) - 6.0*I*F2p*Q*r*exp(-2*I*phi)) + h_-2*(6.0*I*F0*r*exp(2*I*phi) + 6.0*F1*Q*r*exp(2*I*phi) + 12.0*I*F2*r*exp(2*I*phi) + 6.0*I*F2p*Q*r*exp(2*I*phi))

In [12]:
jphi_berlin = (-6*sp.sqrt(2)*jphi_over_Bw2).subs(write_in_hel_basis).expand().rewrite(sp.exp).expand().powsimp().collect([h_p2, h_m2]).subs(berlin)
jphi_berlin

h_+2*(-6.0*F0*r*exp(-2*I*phi) + 6.0*I*F1*Q*r*exp(-2*I*phi) - 12.0*F2*r*exp(-2*I*phi) - 6.0*F2p*Q*r*exp(-2*I*phi)) + h_-2*(-6.0*F0*r*exp(2*I*phi) + 6.0*I*F1*Q*r*exp(2*I*phi) - 12.0*F2*r*exp(2*I*phi) - 6.0*F2p*Q*r*exp(2*I*phi))

The F-factors should have matched Berlin eq 25:

In [13]:
(jr_berlin.coeff(h_p2)/(1j*r*sp.exp(-2j*phi))).subs(expand_fq).expand().collect([Q, Q*Q, Q*Q*Q])

3.0 + (6.0*I - 12.0*I*exp(-1.0*I*Q))/Q - 12.0*exp(-1.0*I*Q)/Q**2 + (-12.0*I + 12.0*I*exp(-1.0*I*Q))/Q**3

In [14]:
(jr_berlin.coeff(h_m2)/(-1j*r*sp.exp(2j*phi))).subs(expand_fq).expand().collect([Q, Q*Q, Q*Q*Q])

3.0 + (6.0*I - 12.0*I*exp(-1.0*I*Q))/Q - 12.0*exp(-1.0*I*Q)/Q**2 + (-12.0*I + 12.0*I*exp(-1.0*I*Q))/Q**3

In [15]:
(jphi_berlin.coeff(h_p2)/(r*sp.exp(-2j*phi))).subs(expand_fq).expand().collect([Q, Q*Q, Q*Q*Q])

3.0 + (6.0*I - 12.0*I*exp(-1.0*I*Q))/Q - 12.0*exp(-1.0*I*Q)/Q**2 + (-12.0*I + 12.0*I*exp(-1.0*I*Q))/Q**3

In [16]:
(jphi_berlin.coeff(h_m2)/(r*sp.exp(2j*phi))).subs(expand_fq).expand().collect([Q, Q*Q, Q*Q*Q])

3.0 + (6.0*I - 12.0*I*exp(-1.0*I*Q))/Q - 12.0*exp(-1.0*I*Q)/Q**2 + (-12.0*I + 12.0*I*exp(-1.0*I*Q))/Q**3

Symbolic Jz

In [17]:
jz_over_Bw2 = ((dmh_ij(2,3,1)-dmh_ij(1,3,2))/omega**2).subs(cart2cyl).expand().collect([h_plus, h_cross])
jz_p = jz_over_Bw2.coeff(h_plus).collect([F2*Q, F2, F2p*Q])
jz_c = jz_over_Bw2.coeff(h_cross).collect([F2*Q, F2, F2p*Q])

In [18]:
jz_p = jz_p.expand().collect([exx_p, eyy_p, exy_p, exz_p, eyz_p, ezz_p])
jz_c = jz_c.expand().collect([exx_c, eyy_c, exy_c, exz_c, eyz_c, ezz_c])

jz_p

e_xx^+*(3*F2*n_y*n_z*r*cos(phi) + F2p*Q*n_y*n_z*r*cos(phi)) + e_xy^+*(-3*F2*n_x*n_z*r*cos(phi) + 3*F2*n_y*n_z*r*sin(phi) - F2p*Q*n_x*n_z*r*cos(phi) + F2p*Q*n_y*n_z*r*sin(phi)) + e_xz^+*(-3*F2*Q*n_y/omega + 3*F2*n_y*n_z*z - F2p*Q**2*n_y/omega + F2p*Q*n_y*n_z*z) + e_yy^+*(-3*F2*n_x*n_z*r*sin(phi) - F2p*Q*n_x*n_z*r*sin(phi)) + e_yz^+*(3*F2*Q*n_x/omega - 3*F2*n_x*n_z*z + F2p*Q**2*n_x/omega - F2p*Q*n_x*n_z*z)

In [19]:
# e_syms_p = [exx_p, exy_p, exz_p, eyy_p, eyz_p, ezz_p]
# e_syms_c = [exx_c, exy_c, exz_c, eyy_c, eyz_c, ezz_c]
# F_monos = [F0, F0p, F1, F1*Q, F2, F2*Q, F2p, F2p*Q, F2p*Q**2]

# def extract_exprs(expr, e_syms, F_monos):
#     expr = sp.expand(expr)
#     out = {}
#     for j, eij in enumerate(e_syms):
#         part = sp.expand(sp.collect(expr, eij).coeff(eij))
#         for k, mono in enumerate(F_monos):
#             c = sp.expand(sp.collect(part, mono).coeff(mono))
#             if c != 0:
#                 out[(j,k)] = c
#     return out

# JR_plus_exprs   = extract_exprs(jr_p,   e_syms_p, F_monos)
# JPHI_plus_exprs = extract_exprs(jphi_p, e_syms_p, F_monos)
# JZ_plus_exprs   = extract_exprs(jz_p,   e_syms_p, F_monos)

# JR_cross_exprs   = extract_exprs(jr_c,   e_syms_c, F_monos)
# JPHI_cross_exprs = extract_exprs(jphi_c, e_syms_c, F_monos)
# JZ_cross_exprs   = extract_exprs(jz_c,   e_syms_c, F_monos)

# import pickle, sympy as sp, numpy as np
# bundle = {
#     "sympy_version": sp.__version__,
#     "numpy_version": np.__version__,
#     "vars": ("r","phi","z","n_x","n_y","n_z","Q","omega"),
#     "F_monos": ["F0", "F0p", "F1","Q*F1","F2","Q*F2","F2p","Q*F2p","Q**2*F2p"],
#     "JR_plus_exprs":   JR_plus_exprs,
#     "JPHI_plus_exprs": JPHI_plus_exprs,
#     "JZ_plus_exprs":   JZ_plus_exprs,
#     "JR_cross_exprs":   JR_cross_exprs,
#     "JPHI_cross_exprs": JPHI_cross_exprs,
#     "JZ_cross_exprs":   JZ_cross_exprs
# }
# with open("/grad/sguotong/projects/CaGe/data/j_eff_expr/j_eff_kernels.pkl","wb") as f:
#     pickle.dump(bundle, f)

In [ ]:
v = n.cross(sp.Matrix([0,0,1]))
v

Matrix([
[ n_y*z - n_z*y],
[-n_x*z + n_z*x],
[ n_x*y - n_y*x]])